# Cutting Cloud Costs with Infrastructure Automation (Part III - Preparing and Training Code on AWS - MNIST)
## Review
At this point, we have a working EC2 instance loaded with the AWS Deep Learning AMI, with the jupyter notebook tested and ready to go.

<img src = "https://media.giphy.com/media/v1HYJrnAMt6gM/giphy.gif" width = "500px">

The last step I want to take to automate this analysis as much as I can is to just get a notebook ready so I can just import it and run it. I'll start with the MNIST data set because I basically already have that code written and tested on my own laptop. All I need to do is to
- Prepare a jupyter notebook
- Host it on git
- Fire up my EC2
- Clone the project and code onto the EC2
- Train on the EC2
- View / save results
- Terminate the EC2 and _**stop the billing**_

Theoretically, this should take no longer than like 5 minutes to set up, and then however long it takes for the model to train (I'm hoping for like less than 30 minutes for 10 epochs, but I honestly have no clue what to expect). I'll use this as the notebook to clone and run on the EC2.

## MNIST Model
Please refer to post \#3 for the fully documented and commentated code. To keep this post light, I'm just going to use the code from post \#3. MNIST should be good example for automation too because it comes with a built in function that loads the data so we don't even have to worry too much about the ETL portion.

### Cost
I just wanted to preface this notebook first by noting how much this exercise will cost me. If things go well, my balance after will only be $0.20 more than it is right now. Current balance:

<img src = "https://s3.ca-central-1.amazonaws.com/2017edmfasatb/chi_lars_face_detection/images/43_aws_billing_before.png" width = "500px">

That exchange rate is killing me... but what can I really do?

<img src = "https://i.giphy.com/media/e3C4pNKkr9rji/giphy.webp" width = "400px">

In [ ]:
# Install tflearn
import os
os.system("pip install tflearn")

In [ ]:
# TFlearn libraries
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import tflearn.datasets.mnist as mnist

# General purpose libraries
import matplotlib.pyplot as plt
import numpy as np
import math

In [ ]:
# Extract data from mnist.load_data()
x, y, x_test, y_test = mnist.load_data(one_hot = True)

In [ ]:
# Reshape x
x_reshaped = x.reshape([-1, 28, 28, 1])
print 'x_reshaped has the shape {}'.format(x_reshaped.shape)

In [ ]:
# Reshape x_test
x_test_reshaped = x_test.reshape([-1, 28, 28, 1])
print 'x_test_reshaped has the shape {}'.format(x_test_reshaped.shape)

In [ ]:
# sentdex's code to build the neural net using tflearn
#   Input layer --> conv layer w/ max pooling --> conv layer w/ max pooling --> fully connected layer --> output layer
convnet = input_data(shape = [None, 28, 28, 1], name = 'input')

convnet = conv_2d(convnet, 32, 2, activation = 'relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation = 'relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation = 'relu')
# convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 10, activation = 'softmax')
convnet = regression(convnet, optimizer = 'sgd', learning_rate = 0.01, loss = 'categorical_crossentropy', name = 'targets')

In [ ]:
model = tflearn.DNN(convnet)
model.fit(
    {'input': x_reshaped}, 
    {'targets': y}, 
    n_epoch = 1, 
    validation_set = ({'input': x_test_reshaped}, {'targets': y_test}), 
    snapshot_step = 500, 
    show_metric = True
)